In [9]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install -U transformers

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install --upgrade 'urllib3==1.26.7' --user

     |████████████████████████████████| 138 kB 27.4 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
sentry-sdk 1.34.0 requires urllib3>=1.26.11; python_version >= "3.6", but you have urllib3 1.26.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
CACHE_DIR = '/moto/edu/engi4800/pb2893/.cache/'

In [1]:
import os
import torch
from datasets import load_dataset,load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    AutoConfig
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



In [23]:
!pip install wandb -qqq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.0.7 which is incompatible.


In [17]:
import wandb
wandb.login()
# run=wandb.init()

True

In [3]:
from transformers import TrainingArguments, Trainer
os.environ["WANDB_PROJECT"] = "capstone_23" # name your W&B project 
os.environ["WANDB_LOG_MODEL"] = "checkpoint" # log all model checkpoints
os.environ["WANDB_DIR"] = "/moto/edu/engi4800/pb2893/.wandb/"
os.environ["WANDB_CACHE_DIR"] = "/moto/edu/engi4800/pb2893/.cache"


In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"
dataset_name = "gbharti/finance-alpaca"

# Fine-tuned model name
new_model = "mistral-7b-finetuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 32

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
!pip install tensorboardX

In [6]:
finance_qa_dataset_name = "gbharti/finance-alpaca"
text_2_sql_dataset_name = "Clinton/Text-to-sql-v1"


In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [22]:
type(dataset)

datasets.arrow_dataset.Dataset

In [23]:
dataset[0]

{'input': '',
 'output': "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 months. The shorter length makes it harder for many to afford. If you can't swing the 12 large payme

In [24]:
dataset = dataset.map(
    lambda example: {"text": 
                     "<s>[INST] " + example["instruction"] + "[/INST]" + example["output"] + "</s>"},
    num_proc=4,
)

In [25]:
dataset = dataset.remove_columns(["input", "instruction", "output"])

In [26]:
dataset = dataset.select(range(1000))

In [27]:
dataset[0]

{'text': "<s>[INST] For a car, what scams can be plotted with 0% financing vs rebate?[/INST]The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 36,48,60 or even 72 months. The shorter length makes i

In [8]:
dataset = load_from_disk('final_dataset')
dataset = dataset.shuffle(seed=42) 

In [9]:
dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [10]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [11]:
# Load base model

model_config = AutoConfig.from_pretrained(
    model_name,
#     use_auth_token=TOKEN, 
    cache_dir=CACHE_DIR
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    config=model_config,
    device_map=device_map, 
    cache_dir = CACHE_DIR
)
model.config.use_cache = True
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [13]:
target_modules=(
"q_proj",
"k_proj",
"v_proj",
"o_proj",
"gate_proj",
"up_proj",
"down_proj",
"lm_head",
),

In [14]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = target_modules
)

In [19]:
# Set training parameters

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to=["wandb"]
)

In [20]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/moto/home/pb2893/.local/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/moto/home/pb2893/.local/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss
25,1.522800
50,1.312800
75,1.462700
100,1.300500
125,1.328800
150,1.304000
175,1.377100
200,1.325600
225,1.309300
250,1.154200


wandb: Adding directory to artifact (./results/checkpoint-25)... Done. 3.7s
wandb: Adding directory to artifact (./results/checkpoint-50)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-75)... Done. 3.7s
wandb: Adding directory to artifact (./results/checkpoint-100)... Done. 3.5s
wandb: Adding directory to artifact (./results/checkpoint-125)... Done. 3.7s
wandb: Adding directory to artifact (./results/checkpoint-150)... Done. 3.5s
wandb: Adding directory to artifact (./results/checkpoint-175)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-200)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-225)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-250)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-275)... Done. 3.6s
wandb: Adding directory to artifact (./results/checkpoint-300)... Done. 3.7s
wandb: Adding directory to artifact (./results/checkpoint-325)... Done. 3.5s
wa

In [21]:
tokenizer.save_pretrained(new_model)

('mistral-7b-finetuned/tokenizer_config.json',
 'mistral-7b-finetuned/special_tokens_map.json',
 'mistral-7b-finetuned/tokenizer.model',
 'mistral-7b-finetuned/added_tokens.json',
 'mistral-7b-finetuned/tokenizer.json')

In [24]:
trainer.save_model('tmp')

In [23]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}